<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#MLND-Final-Project---Memory-Optimization" data-toc-modified-id="MLND-Final-Project---Memory-Optimization-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>MLND Final Project - Memory Optimization</a></span></li><li><span><a href="#A-peek-into-data" data-toc-modified-id="A-peek-into-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>A peek into data</a></span></li><li><span><a href="#Data-Engineering" data-toc-modified-id="Data-Engineering-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Engineering</a></span><ul class="toc-item"><li><span><a href="#Analyze-data-type" data-toc-modified-id="Analyze-data-type-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Analyze data type</a></span></li><li><span><a href="#Convert-data-type" data-toc-modified-id="Convert-data-type-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Convert data type</a></span></li><li><span><a href="#Whole-training-set" data-toc-modified-id="Whole-training-set-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Whole training set</a></span></li><li><span><a href="#Fast-loading-and-retrieving-data-frame" data-toc-modified-id="Fast-loading-and-retrieving-data-frame-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Fast loading and retrieving data frame</a></span></li></ul></li><li><span><a href="#Result" data-toc-modified-id="Result-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Result</a></span></li></ul></div>

# MLND Final Project - Memory Optimization

***This notebook aims at working out a solution for 1) loading dataset into memory at a manageable size 2) Fast retrieving dataframe from disk. ***

# A peek into data 

In [1]:
import numpy as np 
import pandas as pd
import feather
import time

In [2]:
# Check the size of the files provided. 
!ls -lh ./data

total 22767048
-rw-r--r--  1 Bato  staff   473M Jan 28 22:28 X_test.feather
-rw-r--r--  1 Bato  staff   473M Jan 28 22:27 X_val.feather
-rw-r--r--@ 1 Bato  staff   476M Jan 22 16:03 df_2017.feather
-rw-r--r--@ 1 Bato  staff   3.7G Jan 28 22:27 df_x_train.feather
-rw-r--r--  1 Bato  staff   167M Jan 28 22:27 df_y_train.feather
-rwxrwxrwx@ 1 Bato  staff    22K Oct 19 11:30 holidays_events.csv
-rwxrwxrwx@ 1 Bato  staff    99K Oct 19 11:30 items.csv
-rwxrwxrwx@ 1 Bato  staff    20K Oct 19 11:30 oil.csv
-rwxrwxrwx@ 1 Bato  staff    39M Oct 19 11:30 sample_submission.csv
-rwxrwxrwx@ 1 Bato  staff   1.4K Oct 19 11:30 stores.csv
-rwxrwxrwx@ 1 Bato  staff   120M Oct 19 11:30 test.csv
-rwxrwxrwx@ 1 Bato  staff   4.7G Oct 19 11:30 train.csv
-rw-r--r--@ 1 Bato  staff   786M Jan 21 22:32 train_raw.feather
-rwxrwxrwx@ 1 Bato  staff   1.5M Oct 19 11:30 transactions.csv
-rw-r--r--  1 Bato  staff    10M Jan 25 17:03 weights.npy
-rw-r--r--  1 Bato  staff    21M Jan 28 22:27 y_val.feather


In [3]:
# Count the number of rows of train.csv and test.csv
!wc -l ./data/train.csv
!wc -l ./data/test.csv

 125497041 ./data/train.csv
 3370465 ./data/test.csv


**Two Problems occur**:

- The file `train.csv` has about 4.7G and a hundred twenty five million rows. Although the RAM of our AWS ec2 instance is about 60G, we still need to do some experiments on local computer or Kaggle's kernel, whose RAM are 8G and 16G respectively. Loading the dataset will take forever, and 4.7G is definitely not going to be a manageable size of local computer. 

- Before we feed data into our model, we need to manipulate and transform the original training set. Given that we might need to do lots of parameter tuning of our deep learning model, we don't want to manipulate on the original dataset every time we run the model. Therefore, we need to think of a way that enable us to save and retrieve the transformed dataset quickly so that we don't waste too much time in data preprocessing. 

# Data Engineering 

## Analyze data type

*Before importing whole training set, let's first play around with a subset of training set, the size of the subset is about one fifth the whole training set. *

Below are the steps we take:

- Check memory consumption of each column. 
- Check the range of values in each column. 
- Check if we can convert datatype and data structure for each column. 

In [4]:
sub_train = pd.read_csv("./data/train.csv", nrows=25497041)

/Users/Bato/anaconda/envs/aind-dog/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
print(sub_train.min())
print("="*35)
print(sub_train.max())
print("="*35)
print(sub_train.dtypes)

id                      0
date           2013-01-01
store_nbr               1
item_nbr            96995
unit_sales          -1344
onpromotion         False
dtype: object
id               25497040
date           2014-06-16
store_nbr              54
item_nbr          1909755
unit_sales          12021
onpromotion          True
dtype: object
id               int64
date            object
store_nbr        int64
item_nbr         int64
unit_sales     float64
onpromotion     object
dtype: object


In [6]:
# Each columns memory comsumption in MB
sub_train_original_memory = sub_train.memory_usage().sum()/(1024*1024)
sub_train.memory_usage(index=True)/(1024*1024)
print("The original memory usage of sub training set is {0} MB".format(sub_train_original_memory))

The original memory usage of sub training set is 1167.1619873046875 MB


>Please check out my [interactive data analysis of supplementary data](https://github.com/Bato803/Sharpest-Mind/blob/master/Intuitive_Interactive_EDA_Plotly_Geojson.ipynb), we know from that analysis there are only 54 stores exists in this dataset. 

In [47]:
# Check value of floating point number

min_value_16 = np.finfo(np.float16).min
max_value_16 = np.finfo(np.float16).max
min_value_32 = np.finfo(np.float32).min
max_value_32 = np.finfo(np.float32).max

print("The range of float16 is between %d and %d", (min_value_16, max_value_16))
print("The range of float32 is between %d and %d", (min_value_32, max_value_32))

The range of float16 is between %d and %d (-65504.0, 65504.0)
The range of float32 is between %d and %d (-3.4028235e+38, 3.4028235e+38)


In [38]:
# The object type of 'date' is string. 
type(sub_train['date'][0])

str

In [7]:
# The 'onpromostion' is actually boolean type variable. 
# But there is lots of nan in it. 
print("If onpromotion has nan in it? Answer: {0}".format(sub_train.isnull().values.any()))
sub_train.tail()

If onpromotion has nan in it? Answer: True


,id,date,store_nbr,item_nbr,unit_sales,onpromotion
25497036,25497036,2014-06-16,45,582863,61.623,False
25497037,25497037,2014-06-16,45,582864,82.156,False
25497038,25497038,2014-06-16,45,582865,50.117,False
25497039,25497039,2014-06-16,45,583795,16.082,False
25497040,25497040,2014-06-16,45,583797,34.520,False


- Column **`store_nbr`**: 

    - There are only 54 stores, but the original dataset is using int64 ($2^{-32}$ - $2^{32}$) to store number. We need to convert it to uint8, who ranges from $0$ to $255$. 

- Column **`Item_nbr`**: 

    - The value of this column ranging from about 96,995 to 1,909,755. The original data type int64 is about 2000 times larger than the maximum value of `item_nbr`. Therefore, int64 is too large for the values in this columns. Instead, uint32 should be a good choic( 0 --- 4,294,967,295 )

- Column **`id`**:  

    - The values range from 125,000,000 to 128,000,000. So, again, uint32 should be a good choice for id as well.
    
- Column **`unit_sales`**:  

    - These values range from $-1344$ to $12021$. Technically speaking, float16 would be enough. But just in case there exist days that might have a large amount of sales (such as Christmas), we use float32 instead. 
    
- Column **`onpromotion`**:

    - This column's values are boolean variable. Therefore, we need to fill in those nan first. Then we convert them into uint8, where 0 represents false and 1 represents true. 






Split date into three columns - 'Year', 'Month', 'Day'. Becase if we look at the table above, date column is consuming the same amount of memory as those whose data type is int64. And if we change it into 3 columns with dtype uint8 or uint16, we could save a lot of memory.

In [11]:
sub_train.tail()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
25497036,25497036,2014-06-16,45,582863,61.623,False
25497037,25497037,2014-06-16,45,582864,82.156,False
25497038,25497038,2014-06-16,45,582865,50.117,False
25497039,25497039,2014-06-16,45,583795,16.082,False
25497040,25497040,2014-06-16,45,583797,34.520,False


## Convert data type

In [48]:
sub_train['store_nbr'] = sub_train['store_nbr'].astype("uint8")
sub_train['item_nbr'] = sub_train['item_nbr'].astype('uint32')
sub_train['id'] = sub_train['id'].astype('uint32')
sub_train['unit_sales'] = sub_train['unit_sales'].astype('float32')

In [49]:
sub_train['onpromotion'].fillna(0, inplace=True)
sub_train['onpromotion'] = sub_train['onpromotion'].apply(lambda x:int(x==True))
sub_train['onpromotion'] = sub_train['onpromotion'].astype('uint8')

In [50]:
# There should be no nan right now. 
print("If onpromotion has nan in it? Answer: {0}".format(sub_train.isnull().values.any()))

If onpromotion has nan in it? Answer: False


In [51]:
new_memory_usage = sub_train.memory_usage().sum()/(1024*1024)

print("Old memory usage is {0} MB, new memory usage is {1} MB, {2} MB is saved".format(sub_train_original_memory, 
                                                                                      new_memory_usage, 
                                                                                      sub_train_original_memory-new_memory_usage))

Old memory usage is 1167.1619873046875 MB, new memory usage is 534.9492855072021 MB, 632.2127017974854 MB is saved


In [54]:
sub_train.memory_usage(index=True)/(1024*1024)

Index            0.000076
id              97.263493
date           194.526985
store_nbr       24.315873
item_nbr        97.263493
unit_sales      97.263493
onpromotion     24.315873
dtype: float64

## Whole training set

We can change the data type while we loading the dataset.

In [14]:
converters = {
        "onpromotion": (lambda p: int(p == "True")),
        "date": (lambda d: pd.datetime.strptime(d, "%Y-%m-%d")) 
    }

dtype_dict={ "id":np.uint32,
            "store_nbr":np.uint8,
            "item_nbr":np.uint32,
            "unit_sales":np.float32,
            # "onpromotion":np.uint32
           }

start_time = time.time()
# Only load in data that's after 2016-04-16
train = pd.read_csv("./data/train.csv", dtype=dtype_dict, converters=converters, skiprows=range(1, 76458909)) 
print("It tooks {0} minutes to run".format((time.time()-start_time)/60))

It tooks 12.468945602575937 minutes to run


In [15]:
print("does onpromotion column has nan in it? Answer: {0}".format(train['onpromotion'].isnull().values.any()))

does onpromotion column has nan in it? Answer: False


- Change type of 'onpromotion' to 'uint8'.
- Only use data started from 2016-8-15
- Calculate the memory usage of the modified dataframe

In [131]:
train['onpromotion'] = train['onpromotion'].astype('uint8')
df_2017 = train.loc[train.date>=pd.datetime(2016,8,15)]
df_2017_memory = df_2017.memory_usage().sum()/(1024*1024)
print("The memory usage of the modified dataframe is {0} MB".format(df_2017_memory))

The memory usage of the modified dataframe is 1071.5914249420166 MB


In [132]:
df_2017.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
11583297,88042205,2016-08-15,1,103665,1.0,0
11583298,88042206,2016-08-15,1,105574,1.0,0
11583299,88042207,2016-08-15,1,105575,19.0,0
11583300,88042208,2016-08-15,1,105577,1.0,0
11583301,88042209,2016-08-15,1,105693,1.0,0


In [133]:
df_2017.tail()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
49038127,125497035,2017-08-15,54,2089339,4.0,0
49038128,125497036,2017-08-15,54,2106464,1.0,1
49038129,125497037,2017-08-15,54,2110456,192.0,0
49038130,125497038,2017-08-15,54,2113914,198.0,1
49038131,125497039,2017-08-15,54,2116416,2.0,0


## Fast loading and retrieving data frame

Feather format enables very fast read and write access for dataframes, read more about it [here](https://blog.rstudio.com/2016/03/29/feather/)

In [136]:
path = "./data/train_raw.feather"
feather.write_dataframe(df_2017, path)

In [137]:
start_time = time.time()
try_load = feather.read_dataframe(path)
print("It tooks {0} seconds to load".format((time.time()-start_time)))

It tooks 2.4868059158325195 seconds to load


In [138]:
try_load.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,88042205,2016-08-15,1,103665,1.0,0
1,88042206,2016-08-15,1,105574,1.0,0
2,88042207,2016-08-15,1,105575,19.0,0
3,88042208,2016-08-15,1,105577,1.0,0
4,88042209,2016-08-15,1,105693,1.0,0


In [139]:
try_load.tail()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
37454830,125497035,2017-08-15,54,2089339,4.0,0
37454831,125497036,2017-08-15,54,2106464,1.0,1
37454832,125497037,2017-08-15,54,2110456,192.0,0
37454833,125497038,2017-08-15,54,2113914,198.0,1
37454834,125497039,2017-08-15,54,2116416,2.0,0


* <span style="color:blue">We can see that, at the very first, it takes 12 minutes to load the dataset. But after saving it at feather format and discarding information from 2016-04-16 to 2016-08-14 , the time needed to load dataset reduced to 2.4 seconds. And that's 300 times faster compared with previous data loading process!</span>

# Result

***From our experiement we knew that, by converting data type we could save around 54% of our memory. Although we can further reduce our memory consumption by spliting 'date' column into 'year', 'month' and 'day' three integer columns, we did not modify the 'date' column because we would utilize the timestamp extensive during feature engineering. ***


***The original training set size is 4.7G, by utilizing our strategy, it ends up at 2.1G. But we do not plan to use all the records back to 2013 to train our model, which will be evaluated on the fourth quarter of 2017. Therefore, we only include data starting at 2016-08-15 to be in our training set. <span style="color:blue">And the size of this part of dataset ends up at 1.1 GB </span>***

***In order not to repeat the same process again and again during training and testing, we save the modified dataframe in feather format, a fast on-disk format for data frames for R and Python, powered by Apache Arrow. By utilizing this technique, <span style="color:blue"> loading in the dataset only takes 2.4 second, which is 300 times faster!</span> ***